In [1]:
from typing import Sequence
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_community.tools import BaseTool
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_core.runnables import Runnable
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain.tools import tool
import random
import os
import requests
from IPython.display import display, Markdown
from dotenv import load_dotenv

In [2]:
load_dotenv()

# Define API Keys from environment
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
SPOTIFY_CLIENT_ID = os.getenv("SPOTIFY_CLIENT_ID")
SPOTIFY_CLIENT_SECRET = os.getenv("SPOTIFY_CLIENT_SECRET")

In [3]:
# Define mood-to-keyword mapping
mood_keywords = {
    "Happy": ["feel good", "happy hits", "good vibes", "sunny day vibes"],
    "Sad": ["sad songs", "tearjerkers", "broken hearts", "emotional ballads"],
    "Angry": ["rage", "hard rock", "metal workout", "angsty teen"],
    "Energetic": ["workout", "party hits", "high energy", "pop pump"],
    "Chill": ["lofi", "chill vibes", "evening acoustics", "lazy day"],
    "Romantic": ["love songs", "romantic hits", "slow dance", "late night feels"],
    "Reflective": ["nostalgia", "deep focus", "indie acoustic", "quiet moments"],
    "Chaotic": ["weird vibes", "genreless", "eclectic", "random noise"]
}

def get_spotify_keyword(mood: str) -> str:
    return random.choice(mood_keywords.get(mood, ["eclectic"]))

In [4]:
@tool(description="Provides humorous responses")
def llm_response(input: str) -> str:
    llm = ChatOpenAI(model="gpt-4o", temperature=0.5, openai_api_key=OPENAI_API_KEY)
    messages = [
        SystemMessage(content="You are a very successful standup comedian. All your responses are extremely humorous."),
        HumanMessage(content=input)
    ]
    return llm.invoke(messages).content

@tool(description="Classifies user's mood from input")
def mood_classifier(input: str) -> str:
    llm = ChatOpenAI(model="gpt-4o", temperature=0.3, openai_api_key=OPENAI_API_KEY)
    system_prompt = (
        "Return ONLY ONE word from this list based on user mood:"
        " Happy, Sad, Angry, Energetic, Chill, Romantic, Reflective, Chaotic."
        " No commentary or punctuation."
    )
    messages = [SystemMessage(content=system_prompt), HumanMessage(content=input)]
    response = llm.invoke(messages).content.strip().capitalize()
    return response if response in mood_keywords else "Reflective"

@tool(description="Fetches a Spotify playlist link")
def get_spotify_playlist(search_term: str) -> str:
    if not SPOTIFY_CLIENT_ID or not SPOTIFY_CLIENT_SECRET:
        return "Missing Spotify credentials."

    token_response = requests.post(
        "https://accounts.spotify.com/api/token",
        data={"grant_type": "client_credentials"},
        auth=(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    )

    if token_response.status_code != 200:
        return "Spotify authentication failed."

    token = token_response.json()["access_token"]
    headers = {"Authorization": f"Bearer {token}"}
    search_response = requests.get(
        "https://api.spotify.com/v1/search",
        headers=headers,
        params={"q": search_term, "type": "playlist", "limit": 5}
    )

    playlists = search_response.json().get("playlists", {}).get("items", [])
    playlists = [p for p in playlists if p and "name" in p and "external_urls" in p and "spotify" in p["external_urls"]]
    if not playlists:
        return f"No playlists found for '{search_term}'"
    playlist = random.choice(playlists)
    return f"Playlist: {playlist['name']}\nLink: {playlist['external_urls']['spotify']}"

In [5]:
def agent(tools: Sequence[BaseTool]) -> Runnable:
    llm = ChatOpenAI(model="gpt-4o", temperature=0, openai_api_key=OPENAI_API_KEY)
    system_prompt = """
    You are a witty assistant. For any user input:
    1. Classify the mood using `mood_classifier`
    2. Map the mood to a search term
    3. Use `get_spotify_playlist` to get a playlist
    4. Respond with the playlist and a humorous closing line using `llm_response`
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])
    return create_openai_tools_agent(llm, tools, prompt)

tools = [llm_response, mood_classifier, get_spotify_playlist]
executor = AgentExecutor(
    agent=agent(tools),
    tools=tools,
    verbose=False,
    handle_parsing_errors=True
)

In [6]:
user_prompt = input("What's on your mind? Tell me your vibe and I’ll get you a playlist!\n")
response = executor.invoke({"input": user_prompt})
display(Markdown(response["output"]))

What's on your mind? Tell me your vibe and I’ll get you a playlist!
I'm really sleepy but can't quite sleep


Here's a playlist to help you drift off: [Sleepy Playlist](https://open.spotify.com/playlist/2k9rqLpkIu0Glr3jvRzOA8). Just make sure you don't accidentally hit shuffle and end up with "Eye of the Tiger" at 3 AM. Nothing like waking up in a cold sweat, ready to fight a kangaroo in your pajamas!